# maplib demo

### Agenda
- Motivation and introduction
- Data frames
- Ontology
- OTTR and mapping
- Datalog reasoning
- Graph explorer
- SHACL

In [1]:
import polars as pl
pl.Config.set_fmt_str_lengths(150)

polars.config.Config

## Data frames

In [2]:
df_planets = pl.read_csv("../data/planets.csv")

In [3]:
print(df_planets.columns)

['planet', 'mass', 'diameter', 'density', 'gravity', 'escape_velocity', 'rotation_period', 'length_of_day', 'distance_from_sun', 'perihelion', 'aphelion', 'orbital_period', 'orbital_velocity', 'orbital_inclination', 'orbital_eccentricity', 'obliquity_to_orbit', 'mean_temperature', 'surface_pressure', 'number_of_moons', 'has_ring_system', 'has_global_magnetic_field']


In [4]:
print(df_planets["planet"])

shape: (9,)
Series: 'planet' [str]
[
	"Mercury"
	"Venus"
	"Earth"
	"Mars"
	"Jupiter"
	"Saturn"
	"Uranus"
	"Neptune"
	"Pluto"
]


In [5]:
# Need to create a planet URI to use as subject
df_planets = df_planets.with_columns( 
    (pl.lit("http://data.treehouse.example/") + pl.col("planet")).alias("planet_uri")
)

In [6]:
print(df_planets.columns)

['planet', 'mass', 'diameter', 'density', 'gravity', 'escape_velocity', 'rotation_period', 'length_of_day', 'distance_from_sun', 'perihelion', 'aphelion', 'orbital_period', 'orbital_velocity', 'orbital_inclination', 'orbital_eccentricity', 'obliquity_to_orbit', 'mean_temperature', 'surface_pressure', 'number_of_moons', 'has_ring_system', 'has_global_magnetic_field', 'planet_uri']


In [7]:
# Select a few columns to work with
df_planets = df_planets.select(
    ["planet", "planet_uri", "mean_temperature", "length_of_day", "orbital_period"]
)

## Mapping

In [8]:
from maplib import Mapping

# Serialise data frame content to RDF using OTTR template
with open("../tpl/tpl-planets.ttl", "r") as file:
    tpl = file.read()
m = Mapping(tpl)

m.expand("http://data.treehouse.example/tpl/Planet", df_planets)

In [9]:
print(m.query(
    """ SELECT ?s ?p ?o 
        WHERE { ?s ?p ?o .}
    """
))


shape: (63, 3)
┌────────────────────────────────┬────────────────────────────────┬────────────────────────────────┐
│ s                              ┆ p                              ┆ o                              │
│ ---                            ┆ ---                            ┆ ---                            │
│ str                            ┆ str                            ┆ str                            │
╞════════════════════════════════╪════════════════════════════════╪════════════════════════════════╡
│ <http://data.treehouse.example ┆ <http://data.treehouse.example ┆ "24.0"^^<http://www.w3.org/200 │
│ /Earth>                        ┆ /lengthOfDay>                  ┆ 1/XMLSchema#double>            │
│ <http://data.treehouse.example ┆ <http://data.treehouse.example ┆ "9.9"^^<http://www.w3.org/2001 │
│ /Jupiter>                      ┆ /lengthOfDay>                  ┆ /XMLSchema#double>             │
│ <http://data.treehouse.example ┆ <http://data.treehouse.example ┆ "24.7"^^